In [1]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

 
import scipy as sp 
import sklearn
import random 
import time 

from sklearn import preprocessing, model_selection


from sklearn.utils import shuffle

In [3]:
#reading data
data1=pd.read_csv("January 2019 new.csv")
data2=pd.read_csv("December.csv")
data3=pd.read_csv("October.csv")
data4=pd.read_csv("November.csv")


In [4]:
#concatenating all data into one dataframe
frames=[data1,data2,data3,data4]
data=pd.concat(frames)


In [5]:
data=data.fillna(1)

In [6]:
#discretizing data for arrival delay
data['bins_delay']=pd.cut(data['ARR_DELAY'],[-130,-20,-10,-5,0,10,2200],
                         labels=["Early by 1 hr","early by 30 mins","early by 10 mins","early by 5 mins","On time","2-48 Hrs"])

In [7]:
data['bins_delay'].value_counts()

early by 30 mins    590704
2-48 Hrs            504102
On time             357618
Early by 1 hr       337843
early by 10 mins    327067
early by 5 mins     262772
Name: bins_delay, dtype: int64

In [8]:
data.head()

,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_NM,DEP_TIME,DEP_DELAY,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,AIR_TIME,bins_delay
0,1,2,9E,5246,MSP,"Minneapolis, MN",MN,Minnesota,MKE,"Milwaukee, WI",WI,Wisconsin,1155.0,33.0,1259.0,8.0,0,0,49.0,On time
1,1,2,9E,5249,ATL,"Atlanta, GA",GA,Georgia,PHF,"Newport News/Williamsburg, VA",VA,Virginia,1002.0,8.0,1132.0,-5.0,0,0,66.0,early by 10 mins
2,1,2,9E,5249,PHF,"Newport News/Williamsburg, VA",VA,Virginia,ATL,"Atlanta, GA",GA,Georgia,1207.0,-6.0,1408.0,-2.0,0,0,89.0,early by 5 mins
3,1,2,9E,5250,MSP,"Minneapolis, MN",MN,Minnesota,CLE,"Cleveland, OH",OH,Ohio,1015.0,10.0,1257.0,-5.0,0,0,79.0,early by 10 mins
4,1,2,9E,5254,MSP,"Minneapolis, MN",MN,Minnesota,RDU,"Raleigh/Durham, NC",NC,North Carolina,1535.0,-5.0,1854.0,-27.0,0,0,114.0,Early by 1 hr


In [9]:
#removing rows that dont give much information
data=data.drop(['ARR_DELAY','CANCELLED','DIVERTED','AIR_TIME','DEP_DELAY','ORIGIN_CITY_NAME','DEST_STATE_NM'],axis=1)

In [10]:
data.head()

,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEP_TIME,ARR_TIME,bins_delay
0,1,2,9E,5246,MSP,MN,Minnesota,MKE,"Milwaukee, WI",WI,1155.0,1259.0,On time
1,1,2,9E,5249,ATL,GA,Georgia,PHF,"Newport News/Williamsburg, VA",VA,1002.0,1132.0,early by 10 mins
2,1,2,9E,5249,PHF,VA,Virginia,ATL,"Atlanta, GA",GA,1207.0,1408.0,early by 5 mins
3,1,2,9E,5250,MSP,MN,Minnesota,CLE,"Cleveland, OH",OH,1015.0,1257.0,early by 10 mins
4,1,2,9E,5254,MSP,MN,Minnesota,RDU,"Raleigh/Durham, NC",NC,1535.0,1854.0,Early by 1 hr


In [11]:
#converting the data into arrays
data=data.values

In [12]:
carrier=LabelEncoder()
data[:,2]=carrier.fit_transform(data[:,2])
origin=LabelEncoder()
data[:,4]=origin.fit_transform(data[:,4])
origin_city=LabelEncoder()
data[:,5]=origin_city.fit_transform(data[:,5])
origin_state=LabelEncoder()
data[:,6]=origin_state.fit_transform(data[:,6])
dest=LabelEncoder()
data[:,7]=dest.fit_transform(data[:,7])
dest_city=LabelEncoder()
data[:,8]=dest_city.fit_transform(data[:,8])
dest_state=LabelEncoder()
data[:,9]=dest_state.fit_transform(data[:,9])

output=LabelEncoder()
data[:,-1]=output.fit_transform(data[:,-1])

In [13]:
#split the data for training the model
from sklearn.model_selection import train_test_split

X=data[:,:-1]
y=data[:,-1]
y=y.astype(int)
y


array([2, 3, 5, ..., 4, 4, 4])

In [14]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.5,random_state=42,shuffle=True)
y_train.shape

(1190053,)

In [15]:
X_train.shape

(1190053, 12)

In [16]:
#creating a Logistic Regression model adn fitting training data
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [17]:
#predicting test data based on the model fit above
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.27


In [18]:
#reading new dataset to test the model
test=pd.read_csv("Full_Data.csv",parse_dates=['Date'])

In [19]:
def convert_date(df):
    df['DAY_OF_WEEK']=df['Date'].dt.dayofweek
    #df['MONTH']=df['Date'].dt.month
    df['DAY_OF_MONTH']=df['Date'].dt.day
    #df['YEAR']=df['Date'].dt.year
    

convert_date(test)

In [20]:
test.head()

,Date,combo,OP_CARRIER_FL_NUM,OP_UNIQUE_CARRIER,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_NM,DEP_TIME,ARR_TIME,DAY_OF_WEEK,DAY_OF_MONTH
0,2020-01-12,"MSP--Minneapolis, MN--MN--CVG--Cincinnati, OH--KY",5919,EV,MSP,"Minneapolis, MN",MN,Minnesota,CVG,"Cincinnati, OH",KY,Kentucky,536,1111,6,12
1,2020-02-04,"MSP--Minneapolis, MN--MN--CVG--Cincinnati, OH--KY",1927,EV,MSP,"Minneapolis, MN",MN,Minnesota,CVG,"Cincinnati, OH",KY,Kentucky,2249,631,1,4
2,2020-03-24,"MSP--Minneapolis, MN--MN--CVG--Cincinnati, OH--KY",5157,YX,MSP,"Minneapolis, MN",MN,Minnesota,CVG,"Cincinnati, OH",KY,Kentucky,552,543,1,24
3,2020-04-03,"MSP--Minneapolis, MN--MN--CVG--Cincinnati, OH--KY",364,F9,MSP,"Minneapolis, MN",MN,Minnesota,CVG,"Cincinnati, OH",KY,Kentucky,1426,558,4,3
4,2020-05-01,"MSP--Minneapolis, MN--MN--CVG--Cincinnati, OH--KY",3290,MQ,MSP,"Minneapolis, MN",MN,Minnesota,CVG,"Cincinnati, OH",KY,Kentucky,254,119,4,1


In [21]:
test=test.drop(['Date','combo','ORIGIN_STATE_NM','DEST_STATE_NM'],axis=1)
test.head()

,OP_CARRIER_FL_NUM,OP_UNIQUE_CARRIER,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEP_TIME,ARR_TIME,DAY_OF_WEEK,DAY_OF_MONTH
0,5919,EV,MSP,"Minneapolis, MN",MN,CVG,"Cincinnati, OH",KY,536,1111,6,12
1,1927,EV,MSP,"Minneapolis, MN",MN,CVG,"Cincinnati, OH",KY,2249,631,1,4
2,5157,YX,MSP,"Minneapolis, MN",MN,CVG,"Cincinnati, OH",KY,552,543,1,24
3,364,F9,MSP,"Minneapolis, MN",MN,CVG,"Cincinnati, OH",KY,1426,558,4,3
4,3290,MQ,MSP,"Minneapolis, MN",MN,CVG,"Cincinnati, OH",KY,254,119,4,1


In [22]:
test=test.values

In [23]:
from sklearn.preprocessing import LabelEncoder

carrier=LabelEncoder()
test[:,1]=carrier.fit_transform(test[:,1])
origin=LabelEncoder()
test[:,2]=origin.fit_transform(test[:,2])
origin_city=LabelEncoder()
test[:,3]=origin_city.fit_transform(test[:,3])
origin_state=LabelEncoder()
test[:,4]=origin_state.fit_transform(test[:,4])
dest=LabelEncoder()
test[:,5]=dest.fit_transform(test[:,5])
dest_city=LabelEncoder()
test[:,6]=dest_city.fit_transform(test[:,6])
dest_state=LabelEncoder()
test[:,7]=dest_state.fit_transform(test[:,7])

In [24]:
test_predictions = logreg.predict(test)

In [25]:
testoutput1=pd.DataFrame({'prediction':test_predictions})
testoutput1['prediction'].value_counts()

0    64801
3     1568
2        8
4        6
1        1
Name: prediction, dtype: int64

In [26]:
test1=pd.read_csv("Full_Data.csv",parse_dates=['Date'])
test1['output']=testoutput1['prediction']

In [27]:
def convert_date(df):
    df['DAY_OF_WEEK']=df['Date'].dt.dayofweek
    df['MONTH']=df['Date'].dt.month
    df['DAY_OF_MONTH']=df['Date'].dt.day
    #df['YEAR']=df['Date'].dt.year
    

convert_date(test1)

In [28]:
test1.to_csv("Full_nost_arr_delay.csv")

In [29]:
#reading new dataset to test the model
test_half=pd.read_csv("Half_Data.csv",parse_dates=['Date'])

In [30]:
def convert_date(df):
    df['DAY_OF_WEEK']=df['Date'].dt.dayofweek
    #df['MONTH']=df['Date'].dt.month
    df['DAY_OF_MONTH']=df['Date'].dt.day
    #df['YEAR']=df['Date'].dt.year
    

convert_date(test_half)

In [31]:
test_half.head()

,Date,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_NM,DEP_TIME,ARR_TIME,DAY_OF_WEEK,DAY_OF_MONTH
0,2020-08-17,EV,3625,MSP,"Minneapolis, MN",MN,Minnesota,CVG,"Cincinnati, OH",KY,Kentucky,1538,916,0,17
1,2020-11-16,WN,5935,MSP,"Minneapolis, MN",MN,Minnesota,MKE,"Milwaukee, WI",WI,Wisconsin,539,1533,0,16
2,2020-12-13,F9,4658,MSY,"New Orleans, LA",LA,Louisiana,LGA,"New York, NY",NY,New York,2229,1728,6,13
3,2020-04-07,MQ,225,PWM,"Portland, ME",ME,Maine,JFK,"New York, NY",NY,New York,517,44,1,7
4,2020-09-16,DL,4119,DTW,"Detroit, MI",MI,Michigan,PIT,"Pittsburgh, PA",PA,Pennsylvania,811,1016,2,16


In [32]:
test_half=test_half.drop(['Date','ORIGIN_STATE_NM','DEST_STATE_NM'],axis=1)
test_half.head()

,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEP_TIME,ARR_TIME,DAY_OF_WEEK,DAY_OF_MONTH
0,EV,3625,MSP,"Minneapolis, MN",MN,CVG,"Cincinnati, OH",KY,1538,916,0,17
1,WN,5935,MSP,"Minneapolis, MN",MN,MKE,"Milwaukee, WI",WI,539,1533,0,16
2,F9,4658,MSY,"New Orleans, LA",LA,LGA,"New York, NY",NY,2229,1728,6,13
3,MQ,225,PWM,"Portland, ME",ME,JFK,"New York, NY",NY,517,44,1,7
4,DL,4119,DTW,"Detroit, MI",MI,PIT,"Pittsburgh, PA",PA,811,1016,2,16


In [33]:
test_half=test_half.values

In [34]:
from sklearn.preprocessing import LabelEncoder

carrier=LabelEncoder()
test_half[:,0]=carrier.fit_transform(test_half[:,0])
origin=LabelEncoder()
test_half[:,2]=origin.fit_transform(test_half[:,2])
origin_city=LabelEncoder()
test_half[:,3]=origin_city.fit_transform(test_half[:,3])
origin_state=LabelEncoder()
test_half[:,4]=origin_state.fit_transform(test_half[:,4])
dest=LabelEncoder()
test_half[:,5]=dest.fit_transform(test_half[:,5])
dest_city=LabelEncoder()
test_half[:,6]=dest_city.fit_transform(test_half[:,6])
dest_state=LabelEncoder()
test_half[:,7]=dest_state.fit_transform(test_half[:,7])


In [35]:
test_predictions = logreg.predict(test_half)

In [36]:
testoutput2=pd.DataFrame({'prediction':test_predictions})
testoutput2['prediction'].value_counts()

0    5349
3     180
2       2
1       1
Name: prediction, dtype: int64

In [ ]:
test2=pd.read_csv("Full_Data.csv",parse_dates=['Date'])
test2['output']=testoutput2['prediction']

In [ ]:
def convert_date(df):
    df['DAY_OF_WEEK']=df['Date'].dt.dayofweek
    df['MONTH']=df['Date'].dt.month
    df['DAY_OF_MONTH']=df['Date'].dt.day
    #df['YEAR']=df['Date'].dt.year
    

convert_date(test2)

In [ ]:
test2.head()

In [ ]:
test2.to_csv("Half_nost_arr_delay.csv")

In [ ]:
a=list(output.inverse_transform([0,1,2,3,4,5]))
b=list(output.classes_)
c=pd.DataFrame({'Class':a,
                'Label':b})
c